In [1]:
import numpy as np
import pandas as pd
import gc
from tqdm import *
import os
import copy
import sys
import time
from collections import Counter, deque
from multiprocessing import Pool, cpu_count

import transforms3d

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import open3d as o3d

import warnings
warnings.filterwarnings("ignore")

In [2]:
pcd_path = "../DeepMapping/data/2D/ref_run_ccw_r_tenth_scan/"

In [3]:
voxel_size = 0.02
max_correspondence_distance_coarse = voxel_size * 15
max_correspondence_distance_fine = voxel_size * 1.5

In [4]:
def load_point_clouds(voxel_size=0.0):
    pcds = []
    for fname in sorted(os.listdir(pcd_path))[:10]:
        pcd = o3d.io.read_point_cloud(os.path.join(pcd_path, fname))
        pcd_down = o3d.geometry.voxel_down_sample(pcd, voxel_size = voxel_size)
        pcds.append(pcd_down)
    return pcds


def pairwise_registration(source, target):
    print("Apply point-to-plane ICP")
    icp_coarse = o3d.registration.registration_icp(
        source, target, max_correspondence_distance_coarse, np.identity(4),
        o3d.registration.TransformationEstimationPointToPlane())
    icp_fine = o3d.registration.registration_icp(
        source, target, max_correspondence_distance_fine,
        icp_coarse.transformation,
        o3d.registration.TransformationEstimationPointToPlane())
    transformation_icp = icp_fine.transformation
    information_icp = o3d.registration.get_information_matrix_from_point_clouds(
        source, target, max_correspondence_distance_fine,
        icp_fine.transformation)
    return transformation_icp, information_icp


def full_registration(pcds, max_correspondence_distance_coarse,
                      max_correspondence_distance_fine):
    pose_graph = o3d.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)
    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            transformation_icp, information_icp = pairwise_registration(
                pcds[source_id], pcds[target_id])
            print("Build o3d.registration.PoseGraph")
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(
                    o3d.registration.PoseGraphNode(np.linalg.inv(odometry)))
                pose_graph.edges.append(
                    o3d.registration.PoseGraphEdge(source_id,
                                                   target_id,
                                                   transformation_icp,
                                                   information_icp,
                                                   uncertain=False))
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.registration.PoseGraphEdge(source_id,
                                                   target_id,
                                                   transformation_icp,
                                                   information_icp,
                                                   uncertain=True))
    return pose_graph

In [6]:
o3d.utility.set_verbosity_level(o3d.utility.VerbosityLevel.Debug)
pcds_down = load_point_clouds(voxel_size)
o3d.visualization.draw_geometries(pcds_down)

print("Full registration ...")
pose_graph = full_registration(pcds_down,
                               max_correspondence_distance_coarse,
                               max_correspondence_distance_fine)

print("Optimizing PoseGraph ...")
option = o3d.registration.GlobalOptimizationOption(
    max_correspondence_distance=max_correspondence_distance_fine,
    edge_prune_threshold=0.25,
    reference_node=0)
o3d.registration.global_optimization(
    pose_graph, o3d.registration.GlobalOptimizationLevenbergMarquardt(),
    o3d.registration.GlobalOptimizationConvergenceCriteria(), option)

print("Transform points and display")
for point_id in range(len(pcds_down)):
    print(pose_graph.nodes[point_id].pose)
    pcds_down[point_id].transform(pose_graph.nodes[point_id].pose)
o3d.visualization.draw_geometries(pcds_down)

print("Make a combined point cloud")
pcds = load_point_clouds(voxel_size)
pcd_combined = o3d.geometry.PointCloud()
for point_id in range(len(pcds)):
    pcds[point_id].transform(pose_graph.nodes[point_id].pose)
    pcd_combined += pcds[point_id]
pcd_combined_down = o3d.geometry.voxel_down_sample(pcd_combined, voxel_size = voxel_size)
o3d.io.write_point_cloud("multiway_registration.pcd", pcd_combined_down)
o3d.visualization.draw_geometries([pcd_combined_down])

Full registration ...
Apply point-to-plane ICP
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.registration.PoseGraph
Apply point-to-plane ICP
Build o3d.registration.Po

In [7]:
pcd = o3d.io.read_point_cloud("./multiway_registration.pcd")

In [8]:
pcd

geometry::PointCloud with 6357 points.

In [10]:
o3d.visualization.draw_geometries([pcd])